In [4]:
!pip install transformers
!pip install torch


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 7.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
pinecone_api_key=""
pinecone_env=""
                 

# DATASET 1

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

doctors_df = pd.read_csv("emirates.csv")
specialities_texts = doctors_df["Specialty"].tolist()


model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def create_embedding(tokens):
    inputs = tokenizer(tokens, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings.flatten().tolist()

array = np.unique(np.array(specialities_texts))
word_embeddings = {}
for word in array:
    word_embedding = create_embedding(word)
    word_embeddings[word] = word_embedding

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
list(word_embeddings.keys())

['Audiologist',
 'Clinical Dietitian',
 'Clinical Nutritionist',
 'Consultant Anaesthetist',
 'Consultant Breast Surgeon',
 'Consultant Cardiac Electrophysiology',
 'Consultant Cardiologist',
 'Consultant Dermatologist',
 'Consultant ENT',
 'Consultant Emergency Medicine',
 'Consultant Endocrinologist',
 'Consultant Gastroenterologist',
 'Consultant General Surgeon',
 'Consultant General Surgeon and Bariatric Surgery',
 'Consultant General Surgeon and Coloproctology',
 'Consultant Infectious Diseases',
 'Consultant Interventional Cardiologist',
 'Consultant Maternal and Fetal Medicine   Obstetrics & Gynaecology',
 'Consultant Neonatologist   Specialist Paediatrician',
 'Consultant Neonatology',
 'Consultant Nephrologist',
 'Consultant Neurosurgeon',
 'Consultant Obstetrics & Gynaecology',
 'Consultant Ophthalmic Surgeon',
 'Consultant Orthopaedic Surgeon',
 'Consultant Paediatrics',
 'Consultant Pediatric Cardiologist',
 'Consultant Pediatric Surgeon',
 'Consultant Pediatrician & Neona

# DATASET 2

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

doctors_df = pd.read_csv("doc.csv")
specialities_texts = doctors_df["Specialty"].tolist()
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


def create_embedding(tokens):
    inputs = tokenizer(tokens, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings.flatten().tolist()

array2 = np.unique(np.array(specialities_texts))

new_array=[]
for entry in array2:
    if '\n' in entry:
        words = entry.split('\n')
        new_array.extend(words)
    else:
        new_array.append(entry)
array2 = np.array(array2)

word_embeddings2 = {}
for word in new_array:
    word_embedding2 = create_embedding(word)
    word_embeddings2[word] = word_embedding2

# Importing and upserting pinecone

In [3]:
import pinecone      

pinecone.init( 
    api_key='c850f8c2-0513-4f55-894f-db37f01655d5',      
    environment='gcp-starter'      
)      
index = pinecone.Index('test')

In [4]:
import pandas as pd
df = pd.read_csv("emirates.csv")
metadata = df.to_dict(orient="records")

In [38]:
metadata[0]

{'Doctor': 'Dr. Abdul Munnon Durrani',
 'Specialty': 'Specialist Urologist',
 'Contact': '800 444 444'}

In [5]:
vectors_to_upsert=[]
for item in metadata:
    embedding = word_embeddings[item['Specialty']]
    vectors_to_upsert.append({"values":embedding,"metadata":item})

In [6]:
len(vectors_to_upsert)

272

In [7]:
vectors_to_upsert[0]

{'values': [-0.08645811676979065,
  0.4379284977912903,
  0.3657039999961853,
  0.1939639151096344,
  -0.9741953611373901,
  -0.8238250017166138,
  0.37119561433792114,
  0.6245988607406616,
  -0.25666192173957825,
  -0.12488710880279541,
  -0.5204395651817322,
  0.23745033144950867,
  -0.37464630603790283,
  0.8436025381088257,
  -1.0960719585418701,
  -0.019751721993088722,
  -0.027839278802275658,
  0.18622460961341858,
  0.3130367696285248,
  -0.6499212980270386,
  -0.4529339671134949,
  0.5893815755844116,
  0.13816110789775848,
  -0.4851469397544861,
  0.059036754071712494,
  -0.721435546875,
  -0.16465923190116882,
  -0.0454779788851738,
  -0.24257545173168182,
  -0.31397050619125366,
  -0.12176696211099625,
  -0.16023316979408264,
  -0.9771501421928406,
  -0.30388396978378296,
  0.23119322955608368,
  -0.17056286334991455,
  -0.4200604557991028,
  0.26334455609321594,
  0.4259461760520935,
  0.29447779059410095,
  0.22394037246704102,
  -0.3589443266391754,
  -0.026518756523728

In [8]:
import pandas as pd
df2 = pd.read_csv("doc.csv")
metadata2 = df2.to_dict(orient="records")

In [9]:
metadata2[0]

{'Doctor': 'Aaed Jalal Mohammed Al Khalidi Dr.',
 'Specialty': 'Ophthalmology',
 'Contact': '052 800 2000'}

In [10]:
vectors_to_upsert2=[]
for item in metadata2:
    embedding = word_embeddings2[item['Specialty']]
    vectors_to_upsert2.append({"values":list(embedding),"metadata":item})
   

In [11]:
len(vectors_to_upsert2)

1074

In [12]:
vectors = vectors_to_upsert + vectors_to_upsert2

In [13]:
len(vectors)

1346

In [14]:
type(vectors)

list

# Appendind ID in my list 


In [15]:
import uuid 
for item in vectors:
    item['id'] = str(uuid.uuid4())
    item.update({'id': item.pop('id')})
    

In [16]:
vectors[0]

{'values': [-0.08645811676979065,
  0.4379284977912903,
  0.3657039999961853,
  0.1939639151096344,
  -0.9741953611373901,
  -0.8238250017166138,
  0.37119561433792114,
  0.6245988607406616,
  -0.25666192173957825,
  -0.12488710880279541,
  -0.5204395651817322,
  0.23745033144950867,
  -0.37464630603790283,
  0.8436025381088257,
  -1.0960719585418701,
  -0.019751721993088722,
  -0.027839278802275658,
  0.18622460961341858,
  0.3130367696285248,
  -0.6499212980270386,
  -0.4529339671134949,
  0.5893815755844116,
  0.13816110789775848,
  -0.4851469397544861,
  0.059036754071712494,
  -0.721435546875,
  -0.16465923190116882,
  -0.0454779788851738,
  -0.24257545173168182,
  -0.31397050619125366,
  -0.12176696211099625,
  -0.16023316979408264,
  -0.9771501421928406,
  -0.30388396978378296,
  0.23119322955608368,
  -0.17056286334991455,
  -0.4200604557991028,
  0.26334455609321594,
  0.4259461760520935,
  0.29447779059410095,
  0.22394037246704102,
  -0.3589443266391754,
  -0.026518756523728

In [78]:
type(vectors[]['values'])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3962457987.py, line 1)

In [17]:
vectors[100]

{'values': [-0.16397573053836823,
  -0.09875461459159851,
  -0.19172190129756927,
  0.10204803198575974,
  -0.569842517375946,
  -0.13058322668075562,
  -0.2575906217098236,
  0.18857789039611816,
  0.08359377086162567,
  -0.08720771223306656,
  -0.5695950984954834,
  0.21982860565185547,
  -0.4284743368625641,
  0.02537965588271618,
  -0.3965388238430023,
  0.25321224331855774,
  0.19015449285507202,
  -0.2510884404182434,
  0.4453989267349243,
  -0.15970760583877563,
  -0.30568504333496094,
  0.1226506158709526,
  0.08560430258512497,
  -0.11218614876270294,
  0.2345035970211029,
  -0.30260950326919556,
  -0.02492341585457325,
  -0.11078968644142151,
  0.35068219900131226,
  0.20051874220371246,
  -0.2367865890264511,
  -0.11927015334367752,
  -0.19293397665023804,
  -0.02874673157930374,
  -0.12737222015857697,
  0.26346835494041443,
  -0.06628787517547607,
  0.16434280574321747,
  0.18012557923793793,
  0.26566994190216064,
  0.28220003843307495,
  -0.19883684813976288,
  -0.398389

In [83]:
vectors[0].keys()

dict_keys(['values', 'metadata', 'id'])

In [18]:
import pinecone      

# pinecone.init( 
#     api_key=pinecone_api_key,      
#     environment=pinecone_env      
# )      
# index = pinecone.Index('pdf-engine')

pinecone.init( 
    api_key='c850f8c2-0513-4f55-894f-db37f01655d5',      
    environment='gcp-starter'      
)      
index = pinecone.Index('test')

upsert_response = index.upsert(
   vectors=vectors[:50],
   namespace='test-namespace-for-pdfs'
)

In [162]:
pinecone.init( 
    api_key='c850f8c2-0513-4f55-894f-db37f01655d5',      
    environment='gcp-starter'      
)      
index = pinecone.Index('test')

upsert_response = index.upsert(
   vectors=vectors[1000:1347],
   namespace='test-namespace'
)

# creating cleaned dataset docfinal as doc



In [91]:
df2 = pd.read_csv("docfinal.csv")
def split_specialties(row):
    specialties = row['Specialty']
    if isinstance(specialties, str):
        return pd.Series(specialties.split('\n'))
    else:
        return pd.Series([specialties])

df_split = df2.apply(split_specialties, axis=1).stack().reset_index(level=1, drop=True).reset_index()
df_split.columns = ['index', 'Specialty']

df_result = pd.merge(df2.drop('Specialty', axis=1), df_split, left_index=True, right_on='index').drop('index', axis=1)
df_result.to_csv("doc.csv")